Чтобы запустить, просто можете поменять дирректории. Если не хочется в колабе это делать, то могу переделать, просто дайте знать

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [0]:
os.chdir('/content/gdrive/My Drive/')

In [0]:
os.getcwd()

'/content/gdrive/My Drive'

Разархивирование основного архива

In [0]:
! tar -xzvf bankiru.tar.gz

TopicModelingFintech/
TopicModelingFintech/vozrozhdenie.tar.gz
TopicModelingFintech/sberbank.tar.gz
TopicModelingFintech/gazprombank.tar.gz
TopicModelingFintech/mkb.tar.gz
TopicModelingFintech/raiffeisen.tar.gz
TopicModelingFintech/rgsbank.tar.gz
TopicModelingFintech/locko-bank.tar.gz
TopicModelingFintech/touchbank.tar.gz
TopicModelingFintech/bcs-bank.tar.gz
TopicModelingFintech/rsb.tar.gz
TopicModelingFintech/modulbank.tar.gz
TopicModelingFintech/fk_otkritie.tar.gz
TopicModelingFintech/minbank.tar.gz
TopicModelingFintech/citibank.tar.gz
TopicModelingFintech/trust.tar.gz
TopicModelingFintech/avangard.tar.gz
TopicModelingFintech/jugra.tar.gz
TopicModelingFintech/binbankcreditcard.tar.gz
TopicModelingFintech/roscap.tar.gz
TopicModelingFintech/absolutbank.tar.gz
TopicModelingFintech/transcapitalbank.tar.gz
TopicModelingFintech/yandexdengi.tar.gz
TopicModelingFintech/crediteuropebank.tar.gz
TopicModelingFintech/ubrr.tar.gz
TopicModelingFintech/homecreditbank.tar.gz
TopicModelingFintech/ren

In [0]:
os.chdir('/content/gdrive/My Drive/TopicModelingFintech')

In [18]:
os.getcwd()

'/content/gdrive/My Drive/TopicModelingFintech'

Разархивирование подархивов

In [0]:
%%bash
for file in $(find )
do
tar -xzf $file
done

tar (child): .: Cannot read: Is a directory
tar (child): At beginning of tape, quitting now
tar (child): Error is not recoverable: exiting now

gzip: stdin: unexpected end of file
tar: Child returned status 2
tar: Error is not recoverable: exiting now


Создание файла, в котором будем хранить названия файлов с текстами (pickls с текстами)

In [0]:
! touch '/content/files.txt'

In [0]:
with open('/content/files.txt', 'w') as file:
  for line in os.listdir('/content/gdrive/My Drive/TopicModelingFintech'):
    file.write(os.path.join(os.getcwd(), line, 'sentences_replies.pkl') + '\n')

Скачиваем эмбединги из диппавлова

In [25]:
! wget 'files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec'

--2019-04-14 23:49:46--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2767469831 (2.6G) [application/octet-stream]
Saving to: ‘ft_native_300_ru_twitter_nltk_word_tokenize.vec’

ft_native_300_ru_tw 100%[===================>]   2.58G  5.04MB/s    in 9m 49s  

2019-04-14 23:59:36 (4.48 MB/s) - ‘ft_native_300_ru_twitter_nltk_word_tokenize.vec’ saved [2767469831/2767469831]



In [0]:
os.chdir('/content')

Дальше я делаю словарь слов из всего корпуса, чтобы загружать в ram только эмбединги для этих слов, так как больше у меня не вмещается ни в колабе, где вообще ограничение есть (12 гб), ни на локальном компухтере, который начинает очень тупить  после загрузки 1.5 млн эмбедингов, кто-нибудь помогите, как это по-нормальному делается

In [0]:
def gen_frame():
    with open('files.txt') as file:
        for line in file:
            if os.path.exists(line.strip('\n')):
                yield pd.read_pickle(line.strip('\n'))['lemmatized']
gen = gen_frame()

In [23]:
vocab = {}
count_frames = 0
try:
    while True:
        frame = next(gen)
        for line in frame:
            for word in line.split():
                if word not in vocab:
                    vocab[word]=np.random.normal(size=(100,))
        count_frames+=1
        if count_frames%5==0:
            clear_output()
            print('loaded {n} frames out of 50'.format(n=count_frames))
except StopIteration:
    print('finished')

loaded 50 frames out of 50
finished


Загружаю только нужные эмбединги

In [27]:
def gen_line_from_file():
  with open('/content/ft_native_300_ru_twitter_nltk_word_tokenize.vec') as file:
    file.readline()
    for line in file:
      line = line.split()
      yield line[0], list(map(float, line[1:]))

g = gen_line_from_file()

def make_vocab_from_file(gen, vocab):
    count=0
    try:
        while True:
            word, emb = next(gen)
            if word in vocab:
                vocab[word] = emb
                count+=1
                if count%1000==0:
                    clear_output()
                    print('Loaded {n} out of {m} embeddings'.format(n=count, m=len(vocab)))
    except StopIteration:
        print('finished')

make_vocab_from_file(g, vocab)

Loaded 52000 out of 97173 embeddings
finished


Получилось, что у нас есть только половина, поэтому нужно обучить свои эмбединги

In [0]:
from gensim.models import FastText

In [0]:
batch_size=30

def streamer():
  try:
    while True:
      frame = next(gen)
      for i in range(len(frame.index)//batch_size):
        inds = frame.index[i:i+batch_size]
        yield frame.loc[inds]
  except StopIteration:
    print('finished training')

stream = streamer()

In [0]:
class Streamer(object):
  def __init__(self, generator):
    self.gen = generator
    
  def __iter__(self):
    return self
    
  def __next__(self):
    return next(self.gen)
  
itor = Streamer(stream)

In [0]:
def hashit(argument=None):
  matrix = np.empty(shape=(97173, 100))
  for index, word in enumerate(vocab.keys()):
    matrix[index] = vocab[word]
  return matrix

In [13]:
model = FastText(sentences=itor, size=100, workers=-1, hs=0, negative=10, ns_exponent=0.75)

finished training


Хотелось использовать эмбединги из диппавлова, как инициализацию фасттекста, но как-то не получилось, я просто не знаю, как, видимо, через функцию hashfnx, но принцип ее работы я как-то пока не понял

Теперь эмбедим предложения средним по всем эмбедингам слов и пускаем в дбскан. Параметры подбирались, чтобы классов было в районе 5, по-моему, это адекватно, может, нет. В дальнейшем это все будет лучше. Для примера данные по 5 банкам, то есть метки отзывов по ним по классам можно получить, запустив код. По определению тематик пока что нормального нет ничего, но скоро будет

In [0]:
with open('/content/gdrive/My Drive/TopicModelingFintech/model_config', 'wb') as file:
  model.save(file)

In [0]:
import re
pattern = re.compile(r'(?<=/content/gdrive/My Drive/TopicModelingFintech/)[A-Za-z0-9-_]*(?=/sentences_replies.pkl)')

def gen_frame_with_name():
    with open('files.txt') as file:
        for line in file:
            if os.path.exists(line.strip('\n')):
                name = re.findall(pattern, line)
                yield pd.read_pickle(line.strip('\n'))['lemmatized'], name[0]
                
gen = gen_frame_with_name()

In [0]:
def gen_embeded_corpus(frame):
  corpus = []
  
  for line in frame:
    line = line.split()
    matrix = np.random.normal(size=(len(line), 100))
    for index, word in enumerate(line):
      if word in model.wv:
        matrix[index] = model.wv[word]
    corpus.append(np.mean(matrix, axis=0))
  
  return np.asarray(corpus)

In [18]:
from sklearn.cluster import DBSCAN
clusterer = DBSCAN(eps=0.01, min_samples=5)

count = 0

try:
  with open(os.path.join('/content', 'labels.csv'), 'w') as file:
    file.write('title,labels\n')
    while True:
      frame, name = next(gen)
    
      output = clusterer.fit_predict(X=gen_embeded_corpus(frame))
    
      file.write(name + ',' + ' '.join(list(map(str, output))) + '\n')
      count+=1
      if count==5:
        raise StopIteration
      clear_output()
      print('loaded {n} frames out of max 50'.format(n=count))
except StopIteration:
  print('finished')

loaded 4 frames out of max 50
finished
